<a href="https://colab.research.google.com/github/Dennieeeee/Medical-Cost-and-Diseases/blob/master/Healthcare_Blood_Pressure_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Source: https://wwwn.cdc.gov/nchs/nhanes/Search/DataPage.aspx?Component=Questionnaire&CycleBeginYear=2017

In [1]:
# Load our modules
import numpy as np 
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as matplot

import re
import sklearn

# Ignore any warnings for cleaner output (not necessary)
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Albumin & Creatinine - Urine
lab = pd.read_csv('https://raw.githubusercontent.com/Dennieeeee/Medical-Cost-and-Diseases/master/Data/ALB_CR_J.csv')
demographic = pd.read_csv('https://raw.githubusercontent.com/Dennieeeee/Medical-Cost-and-Diseases/master/Data/DEMO_J.csv')
# blood pressure
examination = pd.read_csv('https://raw.githubusercontent.com/Dennieeeee/Medical-Cost-and-Diseases/master/Data/BPX_J.csv')
# Quesionnaire - Blood Pressure and Cholesterol
question = pd.read_csv('https://raw.githubusercontent.com/Dennieeeee/Medical-Cost-and-Diseases/master/Data/BPQ_J.csv')

# Dietary Interview - Total Nutrient Intakes, First Day
#dietary = pd.read_csv('https://raw.githubusercontent.com/Dennieeeee/Medical-Cost-and-Diseases/master/Data/DR1TOT_J.csv')


demographic.drop(['SEQN','Unnamed: 0'], axis=1, inplace=True)
examination.drop(['SEQN', 'Unnamed: 0'], axis=1, inplace=True)
question.drop(['SEQN', 'Unnamed: 0'], axis=1, inplace=True)
lab.drop(['Unnamed: 0'], axis=1, inplace=True)


In [3]:
df = pd.concat([lab, demographic], axis=1, join='inner')
df = pd.concat([df, examination], axis=1, join='inner')
df = pd.concat([df, question], axis=1, join='inner')


In [4]:
demographic.columns

Index(['SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDAGEMN', 'RIDRETH1',
       'RIDRETH3', 'RIDEXMON', 'RIDEXAGM', 'DMQMILIZ', 'DMQADFC', 'DMDBORN4',
       'DMDCITZN', 'DMDYRSUS', 'DMDEDUC3', 'DMDEDUC2', 'DMDMARTL', 'RIDEXPRG',
       'SIALANG', 'SIAPROXY', 'SIAINTRP', 'FIALANG', 'FIAPROXY', 'FIAINTRP',
       'MIALANG', 'MIAPROXY', 'MIAINTRP', 'AIALANGA', 'DMDHHSIZ', 'DMDFMSIZ',
       'DMDHHSZA', 'DMDHHSZB', 'DMDHHSZE', 'DMDHRGND', 'DMDHRAGZ', 'DMDHREDZ',
       'DMDHRMAZ', 'DMDHSEDZ', 'WTINT2YR', 'WTMEC2YR', 'SDMVPSU', 'SDMVSTRA',
       'INDHHIN2', 'INDFMIN2', 'INDFMPIR'],
      dtype='object')

In [5]:
lab.columns

Index(['SEQN', 'URXUMA', 'URXUMS', 'URDUMALC', 'URXUCR', 'URXCRS', 'URDUCRLC',
       'URDACT'],
      dtype='object')

In [6]:
examination.columns

Index(['PEASCCT1', 'BPXCHR', 'BPAARM', 'BPACSZ', 'BPXPLS', 'BPXPULS', 'BPXPTY',
       'BPXML1', 'BPXSY1', 'BPXDI1', 'BPAEN1', 'BPXSY2', 'BPXDI2', 'BPAEN2',
       'BPXSY3', 'BPXDI3', 'BPAEN3', 'BPXSY4', 'BPXDI4', 'BPAEN4'],
      dtype='object')

In [7]:
question.columns

Index(['BPQ020', 'BPQ030', 'BPD035', 'BPQ040A', 'BPQ050A', 'BPQ080', 'BPQ060',
       'BPQ070', 'BPQ090D', 'BPQ100D'],
      dtype='object')

In [8]:
df.describe()

,SEQN,URXUMA,URXUMS,URDUMALC,URXUCR,URXCRS,URDUCRLC,URDACT,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,DMQADFC,DMDBORN4,DMDCITZN,DMDYRSUS,DMDEDUC3,DMDEDUC2,DMDMARTL,RIDEXPRG,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,...,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR,PEASCCT1,BPXCHR,BPAARM,BPACSZ,BPXPLS,BPXPULS,BPXPTY,BPXML1,BPXSY1,BPXDI1,BPAEN1,BPXSY2,BPXDI2,BPAEN2,BPXSY3,BPXDI3,BPAEN3,BPXSY4,BPXDI4,BPAEN4,BPQ020,BPQ030,BPD035,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D
count,6161.000000,5927.000000,5927.000000,5927.0,5927.000000,5927.000000,5927.000000,5927.000000,6161.0,6161.000000,6161.000000,6161.000000,403.000000,6161.000000,6161.000000,5818.000000,2284.000000,4033.000000,376.000000,6161.000000,6158.000000,1315.000000,1505.000000,3731.000000,3731.000000,731.000000,6161.000000,6161.000000,6161.000000,5856.000000,5856.000000,5856.000000,4461.000000,4461.000000,4461.000000,3321.000000,6161.000000,6161.000000,6161.000000,6161.000000,...,5853.000000,6030.000000,3174.000000,6161.000000,6161.000000,6161.000000,6161.000000,5840.000000,5856.000000,5338.000000,314.000000,1092.000000,4822.000000,4819.000000,4773.000000,5865.000000,4773.000000,4764.000000,4434.000000,4434.000000,4434.000000,4636.000000,4636.000000,4636.000000,4627.000000,4627.000000,4627.000000,407.000000,407.000000,407.000000,6161.000000,2137.000000,2137.000000,2137.00000,1942.000000,6161.000000,4193.000000,4543.000000,4543.000000,1653.000000
mean,97278.515501,47.676970,47.676970,0.0,124.945763,11045.205483,0.000169,44.259091,10.0,1.944327,1.509333,34.663853,10.307692,3.232592,3.497646,1.516157,107.088004,1.910984,1.481383,1.254017,1.118707,9.865399,6.418605,3.520504,2.701689,2.024624,1.103392,1.661906,1.966402,1.097336,1.998634,1.970458,1.073526,1.992827,1.976239,1.132791,3.701996,3.579289,0.495699,0.864308,...,2.048180,1.465672,2.123503,34577.655639,34686.475479,1.515988,141.025483,12.336986,12.113388,2.364936,3.156051,108.888278,1.005185,3.701183,73.687408,1.029668,1.005028,147.232578,121.436626,67.798827,1.999549,121.806299,68.264021,1.971311,121.458396,68.232980,1.966069,129.749386,73.218673,1.749386,1.664503,1.232101,76.300889,1.10014,1.150360,1.725369,1.730980,1.679947,1.655734,1.298246
std,2068.489800,328.050959,328.050959,0.0,84.083273,7432.961348,0.012989,319.427543,0.0,0.229307,0.499953,25.598945,7.030114,1.280707,1.705521,0.499782,71.399505,0.331484,0.574734,1.892975,0.467947,19.677444,5.653108,1.243663,3.297776,0.346325,0.304495,0.473099,0.180208,0.296440,0.036939,0.169335,0.261027,0.084400,0.152322,0.388246,1.718542,1.761766,0.803416,1.058252,...,0.657572,0.715257,0.691579,41578.866111,43756.573864,0.499785,4.194519,17.006204,17.018595,1.598702,0.522202,22.535650,0.071825,0.790629,12.572786,0.169683,0.070739,20.889732,20.200332,16.297736,0.021236,20.734839,16.129498,0.166948,20.592261,16.145046,0.181072,25.999737,17.119943,0.433900,0.560254,0.683157,165.396416,0.39577,0.357516,0.744604,1.719232,1.030431,0.616729,0.642504
min,93705.000000,0.300000,0.300000,0.0,3.540000,312.900000,0.000000,0.270000,10.0,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,2571.068712,0.000000,1.000000,134.000000,1.000000,1.000000,0.000000,2.000000,64.000000,1.000000,2.000000,34.000000,1.000000,1.000000,100.000000,72.000000,0.000000,1.000000,72.000000,0.000000,1.000000,72.000000,0.000000,1.000000,74.000000,0.000000,1.000000,1.000000,1.000000,12.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,95493.000000,4.900000,4.900000,0.0,61.000000,5392.400000,0.000000,5.240000,10.0,2.000000,1.000000,11.000000,4.000000,3.000000,3.000

In [9]:
df.head(3)

,SEQN,URXUMA,URXUMS,URDUMALC,URXUCR,URXCRS,URDUCRLC,URDACT,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,DMQADFC,DMDBORN4,DMDCITZN,DMDYRSUS,DMDEDUC3,DMDEDUC2,DMDMARTL,RIDEXPRG,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,...,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR,PEASCCT1,BPXCHR,BPAARM,BPACSZ,BPXPLS,BPXPULS,BPXPTY,BPXML1,BPXSY1,BPXDI1,BPAEN1,BPXSY2,BPXDI2,BPAEN2,BPXSY3,BPXDI3,BPAEN3,BPXSY4,BPXDI4,BPAEN4,BPQ020,BPQ030,BPD035,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D
0,93705.0,3.2,3.2,0.0,24.0,2121.6,0.0,13.33,10,2,2,2,NaN,5,6,2.0,27.0,NaN,NaN,1,1.0,NaN,NaN,NaN,NaN,NaN,1,1,2,1.0,2.0,2.0,NaN,NaN,NaN,NaN,5,5,3,0,...,3.0,1.0,3.0,9246.491865,8539.731348,2,145,15.0,15.0,5.00,NaN,120.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,50.0,1.0,1.0,2,1.0,2.0,2.0,NaN
1,93706.0,39.6,39.6,0.0,69.0,6099.6,0.0,57.39,10,2,1,2,NaN,3,3,1.0,33.0,NaN,NaN,1,1.0,NaN,NaN,NaN,NaN,NaN,1,1,2,1.0,2.0,2.0,NaN,NaN,NaN,NaN,4,4,2,0,...,3.0,1.0,2.0,37338.768343,42566.614750,1,143,15.0,15.0,5.00,NaN,114.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,2,1.0,2.0,2.0,NaN
2,93707.0,41.6,41.6,0.0,209.0,18475.6,0.0,19.90,10,2,2,66,NaN,4,4,2.0,NaN,2.0,NaN,1,1.0,NaN,NaN,2.0,3.0,NaN,1,2,2,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1,1,0,0,...,1.0,2.0,NaN,8614.571172,8338.419786,2,145,3.0,3.0,0.82,NaN,NaN,1.0,4.0,52.0,1.0,1.0,220.0,NaN,NaN,NaN,NaN,NaN,NaN,202.0,62.0,2.0,198.0,74.0,2.0,1,1.0,50.0,1.0,1.0,1,NaN,1.0,1.0,1.0


In [10]:
df.dropna(axis=1, how='all')
df.dropna(axis=0, how='all')

df = df.rename(columns = {'SEQN' : 'ID',
                          'RIAGENDR' : 'Gender',
                          'DMDYRSUS' : 'Years_in_US', # Nan -> american i guess
                          'INDFMPIR' : 'Family_income',
                          })

df = df.loc[:, ['ID', 'Gender', 'Years_in_US', 'Family_income']]

df.describe()

,ID,Gender,Years_in_US,Family_income
count,6161.000000,6161.000000,1315.000000,5338.000000
mean,97278.515501,1.509333,9.865399,2.364936
std,2068.489800,0.499953,19.677444,1.598702
min,93705.000000,1.000000,1.000000,0.000000
25%,95493.000000,1.000000,4.000000,1.040000
50%,97261.000000,2.000000,6.000000,1.910000
75%,99069.000000,2.000000,7.000000,3.690000
max,100881.000000,2.000000,99.000000,5.000000
